In [ ]:
#export
from local.test import *
from local.basics import *
from local.callback.progress import *

In [ ]:
from local.notebook.showdoc import *

In [ ]:
#default_exp callback.wandb

# Wandb

> Integration with [wandb](https://www.wandb.com/) 

In [ ]:
#export
import wandb
from local.callback.tracker import TrackerCallback

In [ ]:
#export
class WandbCallback(TrackerCallback):
    "Saves model topology, losses & metrics"
    # Record if watch has been called previously (even in another instance)
    _watch_called = False

    def __init__(self, log="gradients", save_model=True, monitor='valid_loss', comp=None, log_preds=True,
                 valid_dl=None, n_preds=36, seed=12345):
        # Check if wandb.init has been called
        if wandb.run is None:
            raise ValueError(
                'You must call wandb.init() before WandbCallback()')
            
        super().__init__(monitor=monitor, comp=comp)
        store_attr(self, 'save_model,log,log_preds,valid_dl,n_preds,seed')
        self.model_path = Path(wandb.run.dir)/'bestmodel.pth'
        self.best = None

    def begin_fit(self):
        "Call watch method to log model topology, gradients & weights"
        super().begin_fit()
        if not WandbCallback._watch_called:
            WandbCallback._watch_called = True
            # Logs model topology and optionally gradients and weights
            wandb.watch(self.learn.model, log=self.log)
            
        if self.log_preds and not self.valid_dl:
            wandbRandom = random.Random(self.seed)  # For repeatability
            self.n_preds = min(self.n_preds, len(self.dbunch.valid_ds))
            idxs = wandbRandom.sample(range(len(self.dbunch.valid_ds)), self.n_preds)
            
            items = [self.dbunch.valid_ds.items[i] for i in idxs]
            test_tls = [tl._new(items, split_idx=1) for tl in self.dbunch.valid_ds.tls]
            self.valid_dl = self.dbunch.valid_dl.new(DataSource(tls=test_tls), bs=self.n_preds)

    def after_epoch(self):
        "Log training loss, validation loss and custom metrics & log prediction samples & save model"
        if self.save_model:
            super().after_epoch()
            if self.new_best:
                with self.model_path.open('wb') as model_file:
                    self.learn.save(model_file)
    
        # Log sample predictions
        if self.log_preds is not None:
            pred_log = []
            b = self.valid_dl.one_batch()
            self.learn.one_batch(0, b)
            preds = getattr(self.loss_func, 'activation', noop)(self.pred)
            out = getattr(self.loss_func, 'decodes', noop)(preds)
            b_out = b[:self.valid_dl.n_inp] + (tuple(out) if is_listy(out) else (out,))
            
            x,y,its    = self.valid_dl._pre_show_batch(b, max_n=self.n_preds)
            x1,y1,outs = self.valid_dl._pre_show_batch(b_out, max_n=self.n_preds)
           
            for (x,y),(_,o) in zip(its,outs):
                # scalar -> likely to be a category
                if isinstance(x, TensorImage): x = x.permute(1,2,0)
                if isinstance(o, (Category, MultiCategory)):
                    pred_log.append(wandb.Image(x, caption=f'Ground Truth: {y}\nPrediction: {o}'))
                # most vision datasets have a "show" function we can use
                elif hasattr(x, "show"):
                    # log input data
                    pred_log.append(wandb.Image(x, caption='Input data', grouping=3))
                    # log label and prediction
                    for im, capt in ((o, "Prediction"),
                                     (y, "Ground Truth")):
                        # Resize plot to image resolution
                        # from https://stackoverflow.com/a/13714915
                        my_dpi = 100
                        fig = plt.figure(frameon=False, dpi=my_dpi)
                        h, w = x.shape[:2]
                        fig.set_size_inches(w / my_dpi, h / my_dpi)
                        ax = plt.Axes(fig, [0., 0., 1., 1.])
                        ax.set_axis_off()
                        fig.add_axes(ax)
                        # Superimpose label or prediction to input image
                        ax = x.show(ctx=ax)
                        ax = im.show(ctx=ax)
                        pred_log.append(wandb.Image(fig, caption=capt))
                        plt.close(fig)
                # likely to be an image
                #elif isinstance(y, (TensorMask, TensorPoint,TensorImage) and (
                #    (len(y.shape) == 2) or
                #    (len(y.shape) == 3 and y.shape[0] in [1, 3, 4])):
                ##    pred_log.extend([
                #        wandb.Image(x, caption='Input data', grouping=3),
                #        wandb.Image(pred[0], caption='Prediction'),
                #        wandb.Image(y, caption='Ground Truth')
                #    ])
                # we just log input data
                else: pred_log.append(wandb.Image(x, caption='Input data'))

            wandb.log({"Prediction Samples": pred_log}, commit=False)

        # Log losses & metrics
        # Adapted from fast.ai "CSVLogger"
        logs = {
            name: stat
            for name, stat in list(
                zip(self.recorder.metric_names, self.recorder.log))
        }
        wandb.log(logs)

    def on_train_end(self, **kwargs):
        "Load the best model."
        if self.save_model:
            # Adapted from fast.ai "SaveModelCallback"
            if self.model_path.is_file():
                with self.model_path.open('rb') as model_file:
                    self.learn.load(model_file)
                    print(f'Loaded best saved model from {self.model_path}.')

Optionally logs weights, gradients, sample predictions and best trained model.

Args:
- log (str): "gradients", "parameters", "all", or None. Losses & metrics are always logged.
- save_model (bool): save model at the end of each epoch. It will also load best model at the end of training.
- monitor (str): metric to monitor for saving best model. None uses default TrackerCallback monitor value.
- mode (str): "auto", "min" or "max" to compare "monitor" values and define best model.
- input_type (str): "images" or None. Used to display sample predictions.
- valid_ds (list): data used for sample predictions if input_type is set.
- n_preds (int): number of predictions to make if input_type is set and validation_data is None.
- seed (int): initialize random generator for sample predictions if input_type is set and validation_data is None.

## Export -

In [ ]:
#hide
from local.notebook.export import *
notebook2script(all_fs=True)

Converted 00_test.ipynb.
Converted 01_core.ipynb.
Converted 01a_utils.ipynb.
Converted 01b_dispatch.ipynb.
Converted 01c_transform.ipynb.
Converted 02_script.ipynb.
Converted 03_torch_core.ipynb.
Converted 03a_layers.ipynb.
Converted 04_dataloader.ipynb.
Converted 05_data_core.ipynb.
Converted 06_data_transforms.ipynb.
Converted 07_data_block.ipynb.
Converted 08_vision_core.ipynb.
Converted 09_vision_augment.ipynb.
Converted 09a_vision_data.ipynb.
Converted 10_pets_tutorial.ipynb.
Converted 11_vision_models_xresnet.ipynb.
Converted 12_optimizer.ipynb.
Converted 13_learner.ipynb.
Converted 13a_metrics.ipynb.
Converted 14_callback_schedule.ipynb.
Converted 14a_callback_data.ipynb.
Converted 15_callback_hook.ipynb.
Converted 15a_vision_models_unet.ipynb.
Converted 16_callback_progress.ipynb.
Converted 17_callback_tracker.ipynb.
Converted 18_callback_fp16.ipynb.
Converted 19_callback_mixup.ipynb.
Converted 20_interpret.ipynb.
Converted 20a_distributed.ipynb.
Converted 21_vision_learner.ipy